In [7]:
from matplotlib.path import Path
import numpy as np
from bluesky.tools.aero import ft, nm, kts, fpm
from bluesky.tools.geo import qdrpos, qdrdist

In [ ]:
def invasion_circle(start_lat, start_lon, start_alt, trk, gs, vs, dtlookahead, area_args):
    """
    Predict the invasion of a circular area.
    """
    
    bottom, top, ctr_lat, ctr_lon, radius = area_args

    # Ray endpoints
    end_lat, end_lon = qdrpos(start_lat, start_lon, trk, gs * dtlookahead / nm)

    # Vectors
    startpoints = np.stack([start_lat, start_lon], axis=1)
    endpoints = np.stack([end_lat, end_lon], axis=1)
    seg_vectors = endpoints - startpoints       # Vectors from start to end point
    seg_vec_norm = np.linalg.norm(seg_vectors, axis=1)      # Norm of the vectors

    # calculate the projection (of the startpoint-center vector) vector on the ray vector
    start_center_vecs = np.array([[ctr_lat, ctr_lon]]) - startpoints
    t_close = np.einsum('ij,ij->i', seg_vectors, start_center_vecs) / seg_vec_norm**2
    t_clamped = np.clip(t_close, 0, 1)  # Clamp t to [0, 1]
    closest_points = startpoints + t_clamped[:, np.newaxis] * seg_vectors  # Closest points on the segment to the center of the area
    _, distance = qdrdist(closest_points[:, 0], closest_points[:, 1], ctr_lat, ctr_lon)
    # Check if the closest point is within the radius of the area
    inside_flag = distance < radius
    # Get t_in and t_out
    virtual_closest = startpoints[inside_flag] + t_close[inside_flag][:, np.newaxis] * seg_vectors[inside_flag]
    _, virtual_distance = qdrdist(virtual_closest[:, 0], virtual_closest[:, 1], ctr_lat, ctr_lon)
    # Use Pythagorean
    _, dist_start_virtual = qdrdist(start_lat[inside_flag], start_lon[inside_flag], virtual_closest[:, 0], virtual_closest[:, 1])
    delta_t = np.sqrt(radius**2 - virtual_distance**2) / dist_start_virtual * t_close[inside_flag]
    t_in = np.clip(t_close[inside_flag] - delta_t, 0, 1)    # Clamp to [0, 1]
    t_out = np.clip(t_close[inside_flag] + delta_t, 0, 1)   # Clamp to [0, 1]
    # Get the altitude at t_in and t_out
    alt_in = (start_alt[inside_flag] + vs[inside_flag] * t_in * dtlookahead) / ft
    alt_out = (start_alt[inside_flag] + vs[inside_flag] * t_out * dtlookahead) / ft
    # Check if the aircraft enters or crosses the area
    below_in = alt_in < bottom
    above_in = alt_in > top
    below_out = alt_out < bottom
    above_out = alt_out > top
    not_crossing = (below_in & below_out) | (above_in & above_out)
    inside_flag[inside_flag] = ~not_crossing

    # debug
    # print(distance, virtual_distance, t_close, delta_t)

    return inside_flag

In [3]:
ctr_lat = 30.0
ctr_lon = 120.0
radius = 5.0
bottom = 6000
top = 10000

area_args = (bottom, top, ctr_lat, ctr_lon, radius)

In [4]:
qdrpos(30, 120, 225, 8)

(np.float64(29.905764983474565), np.float64(119.89134135729815))

ACF 1: 10NM 180DEG from center, 360/3000/250  
ACF 2: 8NM 225DEG from center, 015/8000/240, DES at -1500

In [41]:
start_lat = np.array([29.833293, 29.905765])
start_lon = np.array([120.0, 119.89134])
start_alt = np.array([3000, 8000]) * ft
trk = np.array([0, 15])
gs = np.array([250, 240]) * kts
vs = np.array([0, -2001]) * fpm

In [48]:
invasion_circle(start_lat, start_lon, start_alt, trk, gs, vs, 180, area_args)

[0.        4.0052114] [0.        4.0052114] [0.80096988 0.60468581] [0.40000203 0.24941441]


array([False, False])

In [45]:
end_lat, end_lon = qdrpos(29.905765, 119.89134, 15, 240 / (3600 / 59))

In [46]:
qdrdist(ctr_lat, ctr_lon, end_lat, end_lon)

(np.float64(-111.84831769320822), np.float64(4.9940234403932156))